<a href="https://colab.research.google.com/github/SonGoku27/Klimat/blob/main/Climat_Neon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ удовлетворенности клиентов. Исследование комфорта при работе климатического оборудования.
### Описание проекта:
Вы работаете аналитиком в международной компании, занимающейся производством и продажами климатического оборудования: систем кондиционирования и вентиляции. На одной из выставок, посвященной климатическому оборудованию, группа инженеров из дружественной компании предоставила результаты своего исследования: данные по использованию климатических систем в офисах разных стран, городов, а также оценочные данные от респондентов по тому, насколько им комфортно было находиться в заданных условиях.                                                
  ** Руководство поставило вам задачу проанализировать полученные данные и предоставить выводы о том, в какой тип климатического оборудования стоит инвестировать больше, чтобы повысить удовлетворенность клиентов созданными условиями в зависимости от региона? И на что стоит обратить внимание при развитии бизнеса в разных регионах? **


In [34]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# импорты для регресионного моделирования
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

df = pd.read_csv('/content/drive/MyDrive/yandex_data/data_1.csv', sep=";")

# Исследование данных на качество и предобработка
# Проведение расчётов и исследований Амир (Origami)


1.Приведение названий столбцов в соответствии с PEP8

In [35]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

2.перевод данных к правильному типу

In [36]:
df['год'] = df['год'].astype('int')

3.обработка пропусков


In [37]:
for i in ['Женский','Мужской','Пол не указан']: # зависит от пола
    df['пол'] = df['пол'].fillna('Пол не указан')
    df.loc[df['пол'] == i, 'рост'] = df['рост'].fillna(df['рост'].describe().median())
    df.loc[df['пол'] == i, 'вес'] = df['вес'].fillna(df['вес'].describe().median())

df['оценка_комфорта'] = df['оценка_комфорта'].fillna(df['оценка_комфорта'].describe().loc['50%'])
for j in ['Кондиционирование', 'Вентиляция', 'Смешанный', 'NA']:# зависит от способа охлаждения
    df.loc[df['способ_охлаждения'] == j, 'оценка_комфорта'] = df['оценка_комфорта'].fillna(df['оценка_комфорта'].describe().median())


4.удаление дубликатов(категориальных данных)

In [38]:
df_no_duplicates = df.drop_duplicates(
    subset=['время_года', 'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения', 'способ_обогрева', 'пол',
            'ощущение_температуры_(bool)', 'предпочтительное_изменение_температуры', 'ощущение_движения_воздуха_(bool)',
            'предпочтительное_изменение_движения_воздуха', 'занавески', 'вентилятор',
            'окно', 'двери', 'отопление'],
    keep=False)

6.Обработка данных на выбросы и адекватность данных - отдельная ячейка

In [39]:
def emissions(zxc):
    for i in ['утепление', 'температура_воздуха_в_помещении', 'скорость_воздуха', 'рост',
              'среднемесячная_температура_на_улице']:
        zxc = zxc[(zxc[i] > zxc[i].quantile(0.01)) & (zxc[i] < zxc[i].quantile(0.99))]
    return zxc
df1 = emissions(df)

Неадекватные данные

In [40]:
df.loc[df['климат'] == 'Субтроп океанич', 'климат'] = 'Субтропический океанический'
df.loc[df['способ_охлаждения'] == 'Тепле', 'способ_охлаждения'] = 'Теплее'
df.loc[df['способ_охлаждения'] == 'Холодн', 'способ_охлаждения'] = 'Холоднее'

Дополнительный категориальный столбец - количество_рекламаций_категории - отдельная ячейка

In [41]:
def qwe(x):
    if x <= 1:
        return 'мало'
    elif x == 2:
        return 'средне'
    elif x > 2:
        return 'много'


df['количество_рекламаций_кат'] = [qwe(i) for i in df['количество_рекламаций']]

дополнительный категориальный столбец - возраст_категории - отдельная ячейка

In [42]:
def zxc(z):
    if z <= 44:
        return 'молодой возраст'
    elif 45 <= z <= 59:
        return 'средний возраст'
    elif z > 60:
        return 'пожилой возраст'


df1['возраст_кат'] = [zxc(i) for i in df1['возраст']]


средний возраст респондентов по полу и стране

In [43]:
print(df1[['страна', 'пол', 'возраст']].groupby(['страна', 'пол']).mean())

                           возраст
страна    пол                     
Австралия Женский        46.219512
          Мужской        42.570093
Индия     Женский        33.800000
          Мужской        37.957746
США       Пол не указан  35.724138


Рассчет средней комфортной температуры в зависимости от возрастной категории - отдельная ячейка

In [44]:
df3 = df1[['температура_воздуха_в_помещении', 'оценка_комфорта', 'возраст_кат']]
df3['оценка_комфорта'] = df3.loc[df3['оценка_комфорта'] == 6, 'оценка_комфорта']
print(df3.dropna().groupby(['возраст_кат', 'оценка_комфорта']).mean())

                                 температура_воздуха_в_помещении
возраст_кат     оценка_комфорта                                 
молодой возраст 6.0                                    26.675000
пожилой возраст 6.0                                    29.300000
средний возраст 6.0                                    25.651852


<ipython-input-44-8c5fa080045e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['оценка_комфорта'] = df3.loc[df3['оценка_комфорта'] == 6, 'оценка_комфорта']


# Проверка гипотез Neon

### Есть ли взаимосвязь между количеством рекламаций и оценкой комфорта?
-  Н0: Взаимосвязь между количеством рекламаций и оценкой комфорта нет.
-  Н1: Взаимосвязи между количеством рекламаций и оценкой комфорта есть.

Есть отрицательная корреляция и она точно не случайна так как p-value = 0.     
Так как взаимосвязь есть отклоняем нулевую гипотезу.

In [45]:
# тест на нормальность
stat, p_value = stats.shapiro(df1['количество_рекламаций'])
print(f"p-value: {p_value:.5f} ")  # данные не нормальные
stat, p_value = stats.shapiro(df1['оценка_комфорта'])
print(f"p-value: {p_value:.5f} ") # данные не нормальные

# Коэффициент корреляции Спирмена
corr, p_value = stats.spearmanr( df1['количество_рекламаций'], df1['оценка_комфорта'])
print(f'Корреляция: {round(corr, 3)}, p-value: {round(p_value, 5)}')


p-value: 0.00000 
p-value: 0.00000 
Корреляция: -0.482, p-value: 0.0


### Проверить гипотезу о том, что средняя оценка комфорта отличается в зависимости от страны
- Н0: Средняя оценка комфорта не отличается в зависимости от страны.
- Н1: Средняя оценка комфорта отличается в зависимости от страны.
  Исходя из этих данных можем отклонить вторую гипотезу и принять первую. Т.е,
  Средняя оценка комфорта действительно отличается в зависимости от странны. Также можем наблюдать значительно низкое среднее значение в США относительно других стран.

In [46]:
print(df1.groupby('страна')['оценка_комфорта'].mean())


groups = [group['оценка_комфорта'].values for name, group in df1.groupby('страна')]
# Применим Kruskel Wallis
f_stat, p_value = stats.kruskal(*groups)
print(f"\n H-число: {round(f_stat,3)}, p-value: {round(p_value, 5)}")


страна
Австралия    5.000000
Индия        5.050314
США          4.229310
Name: оценка_комфорта, dtype: float64

 H-число: 38.896, p-value: 0.0


Проверить гипотезу "Влияет ли способ охлаждения на оценку комфорта?"
* Н0: способ охлаждения не влияет на оценку комфорта.
* Н1: способ охлаждения влияет на оценку комфорта

Результаты отклоняют нулевую гипотезу , способ охлаждения влияет на оценку комфорта, но различия минимальны .

In [47]:
# тест на нормальность
for name, group in df1.groupby('способ_охлаждения'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Способ охлаждения: {name}, p-value = {p:.5f}") # данные не нормальные

# Применим Краскел Уолис
groups = [group['оценка_комфорта'].values for name, group in df.groupby('способ_охлаждения')]
h_stat, p_value = stats.kruskal(*groups)
print(f"\nH-статистика: {f_stat:.3f}, p-value: {p_value:.5f}")

Способ охлаждения: Вентиляция, p-value = 0.00002
Способ охлаждения: Кондиционирование, p-value = 0.00000
Способ охлаждения: Смешанный, p-value = 0.00000

H-статистика: 38.896, p-value: 0.00240


Проверить гипотезу "Влияет ли пол на оценку комфорта?"
* Н0: пол не влияет на оценку комфорта.
* Н1: пол влияет на оценку комфорта

Результаты не дают нам оснований отклонять нулевую гипотезу. Следовательно пол не влияет на оценку комфорта.

In [48]:
# тест на нормальность
for name, group in df1.groupby('пол'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Пол: {name}, p-value = {p:.5f}") # данные не нормальные

# Применим метод Манна Уитни
group_male = df1[df1['пол'] == 'Мужской']['оценка_комфорта'].values
group_female = df1[df1['пол'] == 'Женский']['оценка_комфорта'].values
stat, p = stats.mannwhitneyu(group_male, group_female, alternative='two-sided')
print(f"U-статистика: {stat:.3f}, p-value: {p:.5f}")


Пол: Женский, p-value = 0.00000
Пол: Мужской, p-value = 0.00000
Пол: Пол не указан, p-value = 0.00096
U-статистика: 15046.500, p-value: 0.15026


Проверить гипотезу "Влияет ли возрастная группа на оценку комфорта?"
* Н0: возрастная группа не влияет на оценку комфорта.
* Н1: возрастная группа влияет на оценку комфорта

Результаты не дают нам оснований отклонять нулевую гипотезу. Следовательно  возрастная группа не влияет на оценку комфорта.

In [49]:
column_names = df1.columns.tolist()
# тест на нормальность
for name, group in df1.groupby('возраст_кат'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Возрастная группа: {name}, p-value = {p:.5f}") # данные не нормальные

# разделение на
young = df1[df1['возраст_кат'] == 'молодой возраст']['оценка_комфорта']
middle = df1[df1['возраст_кат'] == 'средний возраст']['оценка_комфорта']
old = df1[df1['возраст_кат'] == 'пожилой возраст']['оценка_комфорта']

# Kruskal–Wallis test
stat, p = stats.kruskal(young, middle, old)
print(f"Kruskal–Wallis H = {stat:.4f}, p-value = {p:.5f}")

Возрастная группа: молодой возраст, p-value = 0.00000
Возрастная группа: пожилой возраст, p-value = 0.00000
Возрастная группа: средний возраст, p-value = 0.00000
Kruskal–Wallis H = 4.4519, p-value = 0.10797


Проверить гипотезу "Проверить гипотезу влияет ли климат на оценку комфорта"
* Н0: климат не влияет на оценку комфорта.
* Н1: климат влияет на оценку комфорта

Результаты дают основание отклонить нулевую гипотезу и принять альтернативную.
Климат влияет на оценку комфорта,

In [50]:
# тест на нормальность
for name, group in df1.groupby('климат'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Климат: {name}, p-value = {p:.5f}") # данные не нормальные

# Применим Kruskil wallis test
groups = [group['оценка_комфорта'].values for _, group in df1.groupby('климат')]
f_stat, p_value = stats.f_oneway(*groups)
print(f"\nF-статистика: {f_stat:.3f}, p-value: {p_value:.5f}" )


Климат: Cубтроп океанич, p-value = nan
Климат: Cубтропический океанический, p-value = 0.00000
Климат: Влажный субтропический муссонный, p-value = 0.00004
Климат: Жаркий полузасушливый, p-value = 0.00000
Климат: Субтропическое высокогорье, p-value = 0.00000
Климат: Тропическая влажная саванна, p-value = 0.00000

F-статистика: 3.094, p-value: 0.00941


<ipython-input-50-dd36ee489a0c>:3: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p = stats.shapiro(group['оценка_комфорта'])


Проверить гипотезу "Проверить гипотезу влияет ли время года на оценку комфорта"
* Н0: время года не влияет на оценку комфорта.
* Н1: время года влияет на оценку комфорта

Результаты (p-value ≥ 0.05) дают основание принять нулевую гипотезу , и показывают что различия статистически не значимы

In [51]:
# тест на нормальность
for name, group in df1.groupby('время_года'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Время года: {name}, p-value = {p:.5f}") # данные не нормальные

# Применим метод краскела уолиса
groups = [group['оценка_комфорта'].values for _, group in df1.groupby('время_года')]
stat, p_value = stats.kruskal(*groups)

print(f"\nKruskal-Wallis H-статистика: {stat:.3f}, p-value: {p_value:.5f}\n")



Время года: Весна, p-value = 1.00000
Время года: Зима, p-value = 0.00000
Время года: Лето, p-value = 0.00000
Время года: Осень, p-value = 1.00000

Kruskal-Wallis H-статистика: 7.607, p-value: 0.05487



/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Проверить гипотезу "Хотят ли мужчины и женщины разного изменения температуры / движения воздуха?"
* Н0: Мужчины и женщины не хотят разного изменения движения воздуха?
* Н1: Мужчины и женщины  хотят разного изменения движения воздуха?              
Результаты p ≥ 0.05  Нет статистически значимой связи между полом и предпочтениями. Придерживаемся нулевой гипотезы


In [52]:
contingency_table = pd.crosstab(df['пол'], df['предпочтительное_изменение_движения_воздуха'])
print(contingency_table)
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print(f"X2 = {chi2:.4f}, p-value = {p:.5f}")

предпочтительное_изменение_движения_воздуха  Без изменений  Больше  Меньше
пол                                                                       
Женский                                                 94      52      12
Мужской                                                157      89       8
Пол не указан                                           40      20      12
X2 = 17.1245, p-value = 0.00183


# Регрессионное моделирование Neon
Построить регрессионную модель температуры воздуха в помещении в зависимости от типа охлаждения и других значимых факторов. Выбор факторов - на усмотрение исполнителя.                                                                    
Выбранные факторы кроме основных:     
 * время_года
 * отопление
 * среднемесячная_температура_на_улице
 * климат                                                                      
  Эти факторы были отобраны потому, что логически должны оказывать влияние на температуру внутри помещения

    
  Первая версия, с помощью линейной регрессии

  

In [53]:
# Выбираем признаки и целевую переменную
X = df1[['способ_охлаждения', 'время_года','отопление', 'среднемесячная_температура_на_улице', 'климат' ] ]
         #'окно', 'двери'
y = df1['температура_воздуха_в_помещении']

numeric_features = ['среднемесячная_температура_на_улице']
categorical_features = [
    'время_года',
    'способ_охлаждения',
    'отопление',
    'климат'
    # 'окно', 'двери'
]



# Препроцессор: OneHotEncoder + StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        # ('num', StandardScaler(), numeric_features)
    ]
)

# Модельный пайплайн: сначала преобразование, потом регрессия
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regression', LinearRegression())
])

# Делим на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение
pipeline.fit(X_train, y_train)

# Предсказание
y_pred = pipeline.predict(X_test)
# Метрики
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)

R²: 0.06657855429455095
RMSE: 7.673248568397801


Вторая версия с помощью RandomForestRegressor

In [54]:
# Выбираем признаки и целевую переменную
X = df1[['способ_охлаждения', 'время_года','отопление', 'среднемесячная_температура_на_улице']]
         #'окно', 'двери'
y = df1['температура_воздуха_в_помещении']

numeric_features = ['среднемесячная_температура_на_улице']
categorical_features = [
    'время_года',
    'способ_охлаждения',
    'отопление',
    #'окно', 'двери'
]

# Преобразование признаков
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first'), categorical_features),
    # ('num', StandardScaler(), numeric_features)
])

# Пайплайн с RandomForest
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
pipeline.fit(X_train, y_train)

# Предсказание
y_pred = pipeline.predict(X_test)

# Оценка модели
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
mae = mean_absolute_error(y_test, y_pred)

# Вывод результатов
print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")


R²: 0.0692
RMSE: 7.6623
MAE: 2.6969


### Как итог этого раздела

С помощью регрессионого моделирования былы созданы две модели. Метрики показывают  результы предсказания модели плохие , и их пока нельзя использовать в реальных ситуациях , т.к средняя ошибка в 7 градусов ужасно высокая. Это в основном из за малого количества данных , при большом количестве факторов . В будущем после сбора более значительных данных , планируется заного переобучить модели и затем использовать их в реальных ситуациях